# Alinhamento de sequências

> De modo a manter este notebook legível, foram utilizadas as funções auxiliares presentes no módulo [auxiliares.py](.\auxiliares.py)
> Estas funções foram devidamente desenvolvidas, documentadas e testadas no notebook (acrescentar aqui)

O alinhamento de sequências é uma ferramenta útil em vários contextos, como é o contexto do processamento de linguagem natural e previsão de escrita (i.e. autocorrect). No contexto da bioinformática, o alinhamento de sequências assume um papel preponderante para a identificação de sequências similares ou domínios conservados (seja em contexto de genómica seja em contexto de proteómica).

Permite-nos fazer assunções e correlações entre sequência, estrutura e função.

Eixstem um conjunto de abordagens a esta temática, praticamente todas envolvendo à comparação das sequências numa matrix e ir comparando elemento a elemento atribuindo valores a ocorrências idênticas (match), não-ocorrências (mismatch) e em certos cenários espaços vazios (gaps).

Em programação este problema pode ser resolvido utilizando técnicas de programação dinâmica que, apesar de eficazes, são muito pouco eficientes e custosas em termos de recursos da máquina.

Surgem então dois algoritmos excelentes para resolver esta problemática, sendo eles:

- O algoritom de Needleman-Wunch - utilizado em contexto de alinhamento global
- Smith-Waterman - como uma implementação do algoritmo Needleman-Wucnh mas com o acréscimo de mais uma camada, que substitui todos os valores negativos por 0, sendo um excelente algoritmo para alinhamento local / refinado.

Apresentamos abaixo as explicações descritivas e prescritivas para estes algoritmos bem como a sua implementação em Python.

## Demonstração 


In [13]:
from alinhamento import alinhar_sequencias

# Sequências de teste
# Assume-se uma penalidade de espaço vazio de -4 - valor passado por defeito às nossas funções

s1 = 'ATTTCTCGCCTC'
s2 = 'ATC'

alinhar_sequencias(s1,s2,'nw')
print()
alinhar_sequencias(s1,s2,'sw')



    -   A   T   T   T   C   T   C   G   C   C   T   C
-   0  -4  -8 -12 -16 -20 -24 -28 -32 -36 -40 -44 -48
A  -4   2  -2  -6 -10 -14 -18 -22 -26 -30 -34 -38 -42
T  -8  -2   4   0  -4  -8 -12 -16 -20 -24 -28 -32 -36
C -12  -6   0   3  -1  -2  -6 -10 -14 -18 -22 -26 -30

    -   A   T   T   T   C   T   C   G   C   C   T   C
-   0   E   E   E   E   E   E   E   E   E   E   E   E
A   C   D   E   E   E   E   E   E   E   E   E   E   E
T   C   C   D   D   D   E   D   E   E   E   E   D   E
C   C   C   C   D   D   D   E   D   E   D   D   E   D


ATTTCTCGCCTC
A---------TC

    -   A   T   T   T   C   T   C   G   C   C   T   C
-   0   0   0   0   0   0   0   0   0   0   0   0   0
A   0   2   0   0   0   0   0   0   0   0   0   0   0
T   0   0   4   2   2   0   2   0   0   0   0   2   0
C   0   0   0   3   1   4   0   4   0   2   2   0   4

    -   A   T   T   T   C   T   C   G   C   C   T   C
-   0   0   0   0   0   0   0   0   0   0   0   0   0
A   0   D   0   0   0   0   0   0   0   0   0   0  

## Anatomia dos algoritmos de alinhamento

### Needleman-Wunch

Blabla o que é, o que faz.
Uma imagem talvez.

Código abaixo:

In [5]:
from auxiliares import print_matrix, reconstroi, score_subst

def needleman_wunch(string_1 : str, string_2 : str, score_subst : int, score_space : int = -4) -> tuple: 
  '''
  Parameters
  ----------
  string_1 : str
    primeira string

  string_2 : str 
    segunda string
  
  score_space : int
    valor penalidade atribuido a espaços vazios
  
  score_subst : int
    função que recebe dois caracteres e devolve o score da substituição de um pelo outro

  Returns
  -------

  tuple
    devolve um tuplo com o score do melhor alinhamento possível e o respetivo alinhamento
    
  '''
  # Garantimos que recebemos uma sequência. Este algoritmo não é só para DNA / aminoácidos por isso não usamos o validar_dna

  assert type(string_1) and type(string_2) == str

  # Acrescentamos os gaps para iniciar a matriz corretamente
  string_1 = '-' + string_1
  string_2 = '-' + string_2

  # Definimos o número de linhas e colunas da nossa matriz, posicionando as nossas strings
  # corretamente
  ncols = len(string_1)
  nlins = len(string_2)

  # Inicializar toda a matriz a zeros
  score = [[0 for gap in range(ncols)] for gap in range(nlins)]

  matriz_traceback = [[0 for gap in range(ncols)] for gap in range(nlins)]

  # Inicializar a primeira linha
  # Atribuí-se o valor de penalidade porque a primeira linha corresponde apenas a alinhamento espaços vazios
  # (ver output para visualizar)
  score[0] = [posicao_coluna * score_space for posicao_coluna, elemento in enumerate(string_1)]

  # Como estamos na primeira linha, estamos a alinhar apenas com espaços vazios
  # pelo que a matriz traceback indicará que devemos sempre caminhar à esquerda
  # (ver output para visualizar)

  matriz_traceback[0] = [0 if posicao_coluna == 0 else 'E' for posicao_coluna, elemento in enumerate(string_1)]


  # Inicializar a primeira coluna
  
  for posicao_linha, elemento in enumerate(string_2):
  
    # A lógica de preenchimento é semelhante à das linhas, mudando apenas 
    # a direção, que será vertical

    # Iteramos sobre o indice correspondente à posição do elemento na linha, indice 0 que corresponde
    # a primeira coluna 
    score[posicao_linha][0] = posicao_linha * score_space
  
    # Conforme mencionado acima, mesma lógica mas na vertical
    matriz_traceback[posicao_linha][0] = 0 if posicao_linha == 0 else 'C'


  # Iniciamos o primeiro ciclo for, que emparelha a segunda string com o a matriz score, correndo
  # linha a linha
  for posicao_linha, (elemento_string_2, linha) in enumerate(zip(string_2, score)):

    # No segundo ciclo for começamos a fazer a comparação dos elementos das strings, coluna a coluna

    for posicao_coluna, (elemento_string_1, valor_score) in enumerate(zip(string_1, linha)):

      
      if posicao_linha > 0 and posicao_coluna > 0: # Parte do 0 pois ignora a posição 0 são gaps

        # Compara os elementos ao longo de toda a matriz atribuindo os scores.

        # Compara diagonal
        diag = score[posicao_linha - 1][posicao_coluna - 1] + score_subst(elemento_string_1, elemento_string_2, score_space)
        
        # Compara à esquerda
        esq  = score[posicao_linha    ][posicao_coluna - 1] + score_space

        # Compara acima
        cima = score[posicao_linha - 1][posicao_coluna    ] + score_space

        # Agrupa os resultados, determina o maior score e atribui uma das direções.
        # usa-se uma string de referência para facilitar a atribuição / ser mais eficiente

        escolhas = [diag, esq, cima]
        direcoes = "DEC"

        # Encontramos o valor maior que corresponderá ao melhor score
        # que levará ao alinhamento ótimo

        valor = max(*escolhas)
        
        # Populamos a matriz score
        score[posicao_linha][posicao_coluna] = valor

        # Convertemos em direção e populamos a matriz traceback
        matriz_traceback[posicao_linha][posicao_coluna] = direcoes[escolhas.index(valor)]

        

  # Recorremos à função print_matrix() para imprimir as nossas matrizes score e traceback

  print_matrix(string_1, string_2, score)
  print_matrix(string_1, string_2, matriz_traceback)

  '''
  Devolvemos o último valor da matriz score (o ponto de partida para fazer o traceback)
  acrescentamos também as sequências reconstruídas partindo da matriz de traceback e usando
  a função reconstroi
  '''

  return score[-1][-1], reconstroi(string_1, string_2, score, matriz_traceback, 'nw')



In [6]:
# Testes de alinhamento

s1 = 'ATCC'
s2 = 'ACG'
needleman_wunch(s1,s2,score_subst)

    -   A   T   C   C
-   0  -4  -8 -12 -16
A  -4   2  -2  -6 -10
C  -8  -2   1   0  -4
G -12  -6  -3   0  -1

    -   A   T   C   C
-   0   E   E   E   E
A   C   D   E   E   E
C   C   C   D   D   D
G   C   C   D   D   D


ATCC
A-CG


(-1, ('ATCC', 'A-CG'))

In [7]:
# Testes de unidade

### Smith Waterman

Blabla o que é o que faz

Código abaixo


In [8]:


def smith_waterman(string_1 : str, string_2 : str, score_subst : int, score_space : int = -4) -> tuple: 
  '''
  Parameters
  ----------
  string_1 : str
    primeira string

  string_2 : str 
    segunda string
  
  score_space : int
    valor penalidade atribuido a espaços vazios
  
  score_subst : int
    função que recebe dois caracteres e devolve o score da substituição de um pelo outro

  Returns
  -------

  tuple
    devolve um tuplo com o score do melhor alinhamento possível e o respetivo alinhamento
    
  '''
  # Garantimos que recebemos uma sequência. Este algoritmo não é só para DNA / aminoácidos por isso não usamos o validar_dna

  assert type(string_1) and type(string_2) == str

  # Acrescentamos os gaps para iniciar a matriz corretamente
  string_1 = '-' + string_1
  string_2 = '-' + string_2

  # Definimos o número de linhas e colunas da nossa matriz, posicionando as nossas strings
  # corretamente
  ncols = len(string_1)
  nlins = len(string_2)

  # Inicializar toda a matriz a zeros
  score = [[0 for gap in range(ncols)] for gap in range(nlins)]
  max_score = max([val for linha in score for val in linha])

  matriz_traceback = [[0 for gap in range(ncols)] for gap in range(nlins)]



  # Iniciamos o primeiro ciclo for, que emparelha a segunda string com o a matriz score, correndo
  # linha a linha
  for posicao_linha, (elemento_string_2, linha) in enumerate(zip(string_2, score)):
    #print('#####linha',posicao_linha,'#####') 
    # No segundo ciclo for começamos a fazer a comparação dos elementos das strings, coluna a coluna

    for posicao_coluna, (elemento_string_1, valor_score) in enumerate(zip(string_1, linha)):
      #print('####coluna',posicao_coluna,'####')
      
      if posicao_linha > 0 and posicao_coluna > 0: # Parte do 0 pois ignora a posição 0 são gaps

        # Compara os elementos ao longo de toda a matriz atribuindo os scores.
        
        # Compara diagonal
        diag = score[posicao_linha - 1][posicao_coluna - 1] + score_subst(elemento_string_1, elemento_string_2)
        #print('diag',diag)
        #print('e1,e2',(elemento_string_1,elemento_string_2))
        
        # Compara à esquerda
        esq  = score[posicao_linha    ][posicao_coluna - 1] + score_space
        #print('esq',esq,'score-space',score_space)

        

        # Compara acima
        cima = score[posicao_linha - 1][posicao_coluna    ] + score_space
        #print('cima',cima)
        

        # Agrupa os resultados, determina o maior score e atribui uma das direções.
        # usa-se uma string de referência para facilitar a atribuição / ser mais eficiente

        escolhas = [diag, esq, cima, 0]
        direcoes = "DEC"

        # Encontramos o valor maior que corresponderá ao melhor score
        # que levará ao alinhamento ótimo

        valor = max(*escolhas)
        #print('valor=',valor)
        
        # Populamos a matriz score
        score[posicao_linha][posicao_coluna] = valor


        # Convertemos em direção e populamos a matriz traceback
        if valor != 0: matriz_traceback[posicao_linha][posicao_coluna] = direcoes[escolhas.index(valor)] 
      
        # else: matriz_traceback[posicao_linha][posicao_coluna] = 0

     

  # Recorremos à função print_matrix() para imprimir as nossas matrizes score e traceback

  print_matrix(string_1, string_2, score)
  print_matrix(string_1, string_2, matriz_traceback)

  '''
  Devolvemos o último valor da matriz score (o ponto de partida para fazer o traceback)
  acrescentamos também as sequências reconstruídas partindo da matriz de traceback e usando
  a função reconstroi
  '''
  # Smith Waterman parte do valor mais alto para a reconstrução, por isso o valor de partida tem de ser o maior da matriz
  # Para isso é preciso "espalmar" a matriz para encontrar o valor mais alto

  max_score = max([val for linha in score for val in linha])
  # print('pontuacao-maxima=',max_score)
  return max_score, reconstroi(string_1, string_2, score, matriz_traceback, 'sw')




In [11]:
s1 = 'ACTGTGCGTCA'
s2 = 'ATC'

s1 = 'ACCGTC'
s2 = 'ATCGCC'


smith_waterman(s1,s2, score_subst)


    -   A   C   C   G   T   C
-   0   0   0   0   0   0   0
A   0   2   0   0   0   0   0
T   0   0   1   0   0   2   0
C   0   0   2   3   0   0   4
G   0   0   0   1   5   1   0
C   0   0   2   2   1   4   3
C   0   0   2   4   1   0   6

    -   A   C   C   G   T   C
-   0   0   0   0   0   0   0
A   0   D   0   0   0   0   0
T   0   0   D   0   0   D   0
C   0   0   D   D   0   0   D
G   0   0   0   D   D   E   0
C   0   0   D   D   C   D   D
C   0   0   D   D   D   0   D


ACCGTC
ATCGCC


(6, ('ACCGTC', 'ATCGCC'))

# Lixo / arquivo


In [10]:
from auxiliares import print_matrix, reconstroi, score_subst

def sw(string_1 : str, string_2 : str, score_space : int, score_subst : int) -> tuple: 
  '''
  Parameters
  ----------
  string_1 : str
    primeira string

  string_2 : str 
    segunda string
  
  score_space : int
    valor penalidade atribuido a espaços vazios
  
  score_subst : int
    função que recebe dois caracteres e devolve o score da substituição de um pelo outro

  Returns
  -------

  tuple
    devolve um tuplo com o score do melhor alinhamento possível e o respetivo alinhamento
    
  '''
  # Garantimos que recebemos uma sequência. Este algoritmo não é só para DNA / aminoácidos por isso não usamos o validar_dna

  assert type(string_1) and type(string_2) == str

  # Acrescentamos os gaps para iniciar a matriz corretamente
  string_1 = '-' + string_1
  string_2 = '-' + string_2

  # Definimos o número de linhas e colunas da nossa matriz, posicionando as nossas strings
  # corretamente
  ncols = len(string_1)
  nlins = len(string_2)

  # Inicializar toda a matriz a zeros
  score = [[0 for gap in range(ncols)] for gap in range(nlins)]

  matriz_traceback = [[0 for gap in range(ncols)] for gap in range(nlins)]

  '''
  Smith Waterman não prevê penalidades e scores negativos passam a 0.
  A matriz traceback para no 0.
  
  # Inicializar a primeira linha
  # Atribuí-se o valor de penalidade porque a primeira linha corresponde apenas a alinhamento espaços vazios
  # (ver output para visualizar)
  score[0] = [posicao_coluna * score_space for posicao_coluna, elemento in enumerate(string_1)]

  # Como estamos na primeira linha, estamos a alinhar apenas com espaços vazios
  # pelo que a matriz traceback indicará que devemos sempre caminhar à esquerda
  # (ver output para visualizar)

  matriz_traceback[0] = [0 if posicao_coluna == 0 else 'E' for posicao_coluna, elemento in enumerate(string_1)]
  '''

  '''
  Mesma lógica que as linhas
  # Inicializar a primeira coluna
  
  for posicao_linha, elemento in enumerate(string_2):
  
    # A lógica de preenchimento é semelhante à das linhas, mudando apenas 
    # a direção, que será vertical

    # Iteramos sobre o indice correspondente à posição do elemento na linha, indice 0 que corresponde
    # a primeira coluna 
    score[posicao_linha][0] = posicao_linha * score_space
  
    # Conforme mencionado acima, mesma lógica mas na vertical
    matriz_traceback[posicao_linha][0] = 0 if posicao_linha == 0 else 'C'
  '''

  # Iniciamos o primeiro ciclo for, que emparelha a segunda string com o a matriz score, correndo
  # linha a linha
  for posicao_linha, (elemento_string_2, linha) in enumerate(zip(string_2, score)):

    # No segundo ciclo for começamos a fazer a comparação dos elementos das strings, coluna a coluna

    for posicao_coluna, (elemento_string_1, valor_score) in enumerate(zip(string_1, linha)):

      
      if posicao_linha > 0 and posicao_coluna > 0: # Parte do 0 pois ignora a posição 0 são gaps

        # Compara os elementos ao longo de toda a matriz atribuindo os scores.

        # Compara diagonal
        diag = score[posicao_linha - 1][posicao_coluna - 1] + score_subst(elemento_string_1, elemento_string_2, score_space)

        #print('e1,e2',(elemento_string_1,elemento_string_2))
        
        # Compara à esquerda
        esq  = score[posicao_linha    ][posicao_coluna - 1] + score_space

        

        # Compara acima
        cima = score[posicao_linha - 1][posicao_coluna    ] + score_space

        

        # Agrupa os resultados, determina o maior score e atribui uma das direções.
        # usa-se uma string de referência para facilitar a atribuição / ser mais eficiente

        escolhas = [diag, esq, cima, 0]
        direcoes = "DEC"

        # Encontramos o valor maior que corresponderá ao melhor score
        # que levará ao alinhamento ótimo

        valor = max(*escolhas)
        #print('valor=',valor)
        
        # Populamos a matriz score
        score[posicao_linha][posicao_coluna] = valor

        # Convertemos em direção e populamos a matriz traceback
        if valor != 0: matriz_traceback[posicao_linha][posicao_coluna] = direcoes[escolhas.index(valor)]

        

  # Recorremos à função print_matrix() para imprimir as nossas matrizes score e traceback

  print_matrix(string_1, string_2, score)
  print_matrix(string_1, string_2, matriz_traceback)

  '''
  Devolvemos o último valor da matriz score (o ponto de partida para fazer o traceback)
  acrescentamos também as sequências reconstruídas partindo da matriz de traceback e usando
  a função reconstroi
  '''

  return score[-1][-1], reconstroi(string_1, string_2, matriz_traceback)


s1 = 'ACCTG'
s2 = 'ACGT'
g = -4

sw(s1,s2,g,score_subst)



    -   A   C   C   T   G
-   0   0   0   0   0   0
A   0   2   0   0   0   0
C   0   0   4   2   0   0
G   0   0   0   3   1   2
T   0   0   0   0   5   1

    -   A   C   C   T   G
-   0   0   0   0   0   0
A   0   D   0   0   0   0
C   0   0   D   D   0   0
G   0   0   0   D   D   D
T   0   0   0   0   D   E



TypeError: reconstroi() missing 2 required positional arguments: 'matriz_traceback' and 'algoritmo'

'''
'''
Função que encontra as coordenadas do valor mais alto da matrix score.

'''
def max_score_loc(scr):
    ms = scr[0][0]
    mi,mj = 0,0
    for linha in range(len(scr)): # itera por cada linha
    
        for coluna in range(len(scr[linha])): # itera por cada item de cada linha 
            if scr[linha][coluna] > ms:
                ms = scr[linha][coluna] # regista o maior
                mi,mj = linha,coluna # regista as coordenadas
    
    return mi,mj


'''

Uma vez que a lógica de traceback dos algoritmos NW e SW é diferente, temos de fazer um upgrade á função
reconstroi, adicionando o tipo de algoritmo.

Abaixo o módulo para SW
'''

elif algoritmo == 'sw':
    mi,mj = max_score_loc(result[0])

    s1a = ''
    s2a = ''

    # i = linha
    # j = coluna

    s1 = '-' + s1
    s2 = '-' + s2
    print(mj,mi,s1[mj],s2[mi])
    while mi > 0 and mj > 0:

        if trace[mi][mj] == 'D':
            s1a = s1[mj] + s1a
            s2a = s2[mi] + s2a

            mi -= 1
            mj -= 1

        elif trace[mi][mj] == 'C':
            s1a = '-' + s1a
            s2a = s2[mi] + s2a

            mi -= 1
            mj -= 1

        elif trace[mi][mj] == 'E':
            s1a = s1[mj] + s1a
            s2a = '-' + s2a

            mi -= 1
            mj -= 1
        
        elif trace[mi][mj] == 0:
            s1a = '-' + s1a
            s2a = '-' + s2a

            mi -= 1
            mj -= 1
        
print(s1a,s2a,sep='\n')'
'''